In [2]:
import pandas as pd
import numpy as np
import pipes2 as pp

df = pd.read_csv('air_train.csv')
df = df.drop(['Unnamed: 0','id'], axis = 1)

df = pp.myPipe.transform(df)

X = df.drop(['satisfaction'], axis = 1)
y = df.satisfaction

X = X - X.mean()
X = X/X.std()


In [3]:
df = pd.read_csv('air_test.csv')
df = df.drop(['Unnamed: 0','id'], axis = 1)

df = pp.myPipe.transform(df)

X_t = df.drop(['satisfaction'], axis = 1)
y_t = df.satisfaction

X_t = X_t - X_t.mean()
X_t = X_t/X_t.std()


In [4]:
X.shape[1]

22

In [6]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()

model.add(layers.Dense(256, activation="relu", input_shape = (X.shape[1],),)) 
#model.add(layers.Dropout(0.4))

model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(256, activation="relu"))

model.add(layers.Dense(1, activation="sigmoid"))

# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [11]:
model = create_model()

In [12]:
history = model.fit(X, y, validation_data = (X_t, y_t), 
                    epochs=100, batch_size=512)

Epoch 1/100
203/203 [==============================] - 2s 7ms/step - loss: 0.4319 - accuracy: 0.9205 - val_loss: 0.2362 - val_accuracy: 0.9439
Epoch 2/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1957 - accuracy: 0.9471 - val_loss: 0.1796 - val_accuracy: 0.9491
Epoch 3/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1550 - accuracy: 0.9539 - val_loss: 0.1445 - val_accuracy: 0.9557
Epoch 4/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1376 - accuracy: 0.9565 - val_loss: 0.1451 - val_accuracy: 0.9500
Epoch 5/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1289 - accuracy: 0.9577 - val_loss: 0.1293 - val_accuracy: 0.9567
Epoch 6/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1223 - accuracy: 0.9586 - val_loss: 0.1237 - val_accuracy: 0.9597
Epoch 7/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1171 - accuracy: 0.9598 - val_loss: 0.1234 - val_accuracy: 0.9582

Epoch 58/100
203/203 [==============================] - 1s 6ms/step - loss: 0.0898 - accuracy: 0.9662 - val_loss: 0.0969 - val_accuracy: 0.9634
Epoch 59/100
203/203 [==============================] - 1s 6ms/step - loss: 0.0870 - accuracy: 0.9670 - val_loss: 0.0957 - val_accuracy: 0.9643
Epoch 60/100
203/203 [==============================] - 1s 6ms/step - loss: 0.0877 - accuracy: 0.9664 - val_loss: 0.0977 - val_accuracy: 0.9638
Epoch 61/100
203/203 [==============================] - 1s 6ms/step - loss: 0.0874 - accuracy: 0.9664 - val_loss: 0.0954 - val_accuracy: 0.9647
Epoch 62/100
203/203 [==============================] - 1s 6ms/step - loss: 0.0875 - accuracy: 0.9663 - val_loss: 0.0971 - val_accuracy: 0.9624
Epoch 63/100
203/203 [==============================] - 1s 6ms/step - loss: 0.0886 - accuracy: 0.9660 - val_loss: 0.0962 - val_accuracy: 0.9638
Epoch 64/100
203/203 [==============================] - 1s 6ms/step - loss: 0.0882 - accuracy: 0.9664 - val_loss: 0.1027 - val_accuracy:

In [13]:
from sklearn.metrics import confusion_matrix, f1_score

In [22]:
pr = model.predict(X_t)>0.5

812/812 [==============================] - 1s 1ms/step


In [23]:
pr

array([[ True],
       [ True],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [24]:
f1_score (y_t, pr)

0.9564866681604303

In [25]:
confusion_matrix(y_t, pr)

array([[14333,   240],
       [  731, 10672]])

In [ ]:
# кросс-валидация

In [26]:
from sklearn.model_selection import StratifiedKFold

In [27]:
def create_model():
    model = models.Sequential()

    model.add(layers.Dense(256, activation="relu", input_shape = (X.shape[1],), 
                       kernel_regularizer=tf.keras.regularizers.l2(0.002),)) 
                       #kernel_regularizer=tf.keras.regularizers.l2(0.002)))
        #model.add(layers.Dropout(0.4))

    model.add(layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.002)))
    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dense(1, activation="sigmoid"))

    # Компиляция модели
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy"])
    
    return model

In [35]:
def train_and_evaluate__model(model, data_train, labels_train, data_test, labels_test):
    
    history = model.fit(data_train, labels_train, validation_data = (data_test, labels_test), 
                        epochs=50, batch_size=512, verbose = 0)
    
    return history.history['accuracy'][-1], history.history['val_accuracy'][-1]


In [37]:
kFold = StratifiedKFold(n_splits=5)
i = 0
for train, test in kFold.split(X, y):
    model = None
    model = create_model()
    i += 1
    acc, v_a = train_and_evaluate__model(model, X[train], y[train], X[test], y[test])
    print ('split ',i, 'acc :', acc, 'val_acc :', v_a)

split  1 acc : 0.9672413468360901 val_acc : 0.9624176025390625
split  2 acc : 0.9670608639717102 val_acc : 0.9627063274383545
split  3 acc : 0.9669165015220642 val_acc : 0.9610220789909363
split  4 acc : 0.9671571254730225 val_acc : 0.9645830392837524
split  5 acc : 0.9662191271781921 val_acc : 0.9635707139968872


In [29]:
X = np.asarray(X)

In [30]:
kFold = StratifiedKFold(n_splits=5)

In [31]:
kFold

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)